# Language Modeling using Ngram

In this Exercise, you are going to use NLTK which is a natural language processing library for python to create a bigram language model and its variation. You will build one model for each of the following type and calculate their perplexity:
- Unigram Model
- Bigram Model
- Bigram Model with Laplace smoothing
- Bigram Model with Interpolation
- Bigram Model with Kneser-ney Interpolation
- Neural LM (optional)



In [123]:
#download corpus
!wget --no-check-certificate https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
!unzip BEST2010.zip

--2023-01-28 14:45:31--  https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip [following]
--2023-01-28 14:45:32--  https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8001::154, 2606:50c0:8002::154, 2606:50c0:8003::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8001::154|:443... connected.
HTTP request sent, awaiting response... ^C
unzip:  cannot find or open BEST2010.zip, BEST2010.zip.zip or BEST2010.zip.ZIP.


In [69]:
#First we import necessary library such as math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
random.seed(999)

BEST2010 is a free Thai NLP dataset by NECTEC usually use as a standard benchmark for various NLP tasks includeing language modeling. BEST2010 is separated into 4 domain article, encyclopedia, news and novel. The data is already  tokenized using '|' as a separator.

For example,

ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น|

In [70]:
# We choose news domain as our dataset
best2010=[]
fp= io.open('BEST2010/news.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    best2010.append(line.strip()[:-1])
fp.close()
all_vocabulary =set()
total_word_count =0
for line in best2010:
    for word in line.split('|'):        
        all_vocabulary.add(word)
        total_word_count+=1

In [71]:
#For simplicity, we assumes that each line is a sentence.
print ('Total sentences in BEST2010 news dataset :\t'+ str(len(best2010)))
print ('Total word counts in BEST2010 news dataset :\t'+ str(total_word_count))
print ('Total vocabulary in BEST2010 news dataset :\t'+ str(len(all_vocabulary)))

Total sentences in BEST2010 news dataset :	30969
Total word counts in BEST2010 news dataset :	1660190
Total vocabulary in BEST2010 news dataset :	35488


We separate out input into 2 sets, train and test data with 70:30 ratio

In [73]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_vocabulary =set()
train_word_count =0
for line in train:
    for word in line.split('|'):        
        train_vocabulary.add(word)
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))
print ('Total vocabuary in BEST2010 news training dataset :\t'+ str(len(train_vocabulary)))
# We will use 1/vocab_size as a default value for unknown word
unk_value = math.pow(len(train_vocabulary),-1)

Total sentences in BEST2010 news training dataset :	21678
Total word counts in BEST2010 news training dataset :	1042797
Total vocabuary in BEST2010 news training dataset :	26240


# IMPORTANT. Use the dataset below to answer questions on Mycourseville

The dataset has been tokenized for you (with the vocabulary above).

Please download the provided vocabulary to confirm that you are using the same vocabulary. **Otherwise, you might get wrong answers.**

In [ ]:
!wget https://www.dropbox.com/s/jajdlqnp5h0ywvo/tokenized_wiki_sample.csv?dl=0 
!wget https://www.dropbox.com/s/n7w7te0zc1vz67n/train_vocabulary.pickle

zsh:1: no matches found: https://www.dropbox.com/s/jajdlqnp5h0ywvo/tokenized_wiki_sample.csv?dl=0
--2023-01-27 22:41:52--  https://www.dropbox.com/s/n7w7te0zc1vz67n/train_vocabulary.pickle
Resolving www.dropbox.com (www.dropbox.com)... 2620:100:6031:18::a27d:5112, 162.125.81.18
Connecting to www.dropbox.com (www.dropbox.com)|2620:100:6031:18::a27d:5112|:443... connected.
HTTP request sent, awaiting response... ^C


In [74]:
import pickle
with open("train_vocabulary.pickle","rb") as f:
  old_vocab = pickle.load(f)

In [75]:
assert train_vocabulary == old_vocab #if nothing shows up, you are good to go

# Unigram

In this section, we will demonstrate how to build a unigram language model <br>
**Important note:** <br>
**\<s\>** = sentence start symbol <br>
**\</s\>** = sentence end symbol 

In [76]:
def getUnigramModel(data):
    model = defaultdict(lambda: 0)
    word_count =0
    for sentence in data:
        sentence +=  u'|</s>' #for unigram model we can always ignore <s>, since p(w0=<s>)=1
        for w1 in sentence.split('|'):
            model[w1] +=1.0
            word_count+=1
    for w1 in model:
        model[w1] = model[w1]/(word_count)
    return model

In [77]:
unigram_model = getUnigramModel(train)

In [10]:
def getLnValue(x):
    if x >0.0:
        return math.log(x)
    else:
        return math.log(unk_value)

In [78]:
#problability of 'นายก'
print(getLnValue(unigram_model[u'นายก']))
#for example, problability of 'นายกรัฐมนตรี' which is an unknown word is equal to
print(getLnValue(unigram_model[u'นายกรัฐมนตรี']))
#problability of 'นายก' 'ได้' 'ให้' 'สัมภาษณ์' 'กับ' 'สื่อ'
prob = getLnValue(unigram_model[u'นายก'])+getLnValue(unigram_model[u'ได้'])+ getLnValue(unigram_model[u'ให้'])+getLnValue(unigram_model[u'สัมภาษณ์'])+getLnValue(unigram_model[u'กับ'])+getLnValue(unigram_model[u'สื่อ'])+getLnValue(unigram_model['</s>'])
print ('Problability of a sentence', math.exp(prob))

-6.551526663995246
-10.175040243058024
Problability of a sentence 5.617210748667918e-18


# Perplexity

In order to compare language model we need to calculate perplexity. In this task you should write a perplexity calculation code for the unigram model. The result perplexity should be around 556.39 and
476.07 on train and test data.

## TODO #0
**Fill your name and ID here** <br>
**Name**: Nuttapol Onton
<br>
**ID**: 6330166021

## TODO #1 **Calculate perplexity**

In [79]:
def calculate_sentence_ln_prob(sentence, model):
    sentence += '|</s>'
    word = sentence.split('|')
    ln_prob = .0
    for w in word:
        ln_prob += getLnValue(model[w])
    return ln_prob, len(word)

def perplexity(test,model):
    ln_prob = .0
    word_count = .0
    for sentence in test:
        ans = calculate_sentence_ln_prob(sentence, model)
        ln_prob += ans[0]
        word_count += ans[1]
    return math.exp(-ln_prob / word_count)

In [80]:
print(perplexity(train,unigram_model))
print(perplexity(test,unigram_model))

556.3925994212195
476.0687892303532


## MyCourseVille Question #1 
How much perplexity do you get with the Unigram model on the tokenized_wiki_sample dataset?

Please also leave the number below.

In [81]:
import pandas as pd

df = pd.read_csv("tokenized_wiki_sample.csv")
wiki_sample_data = df["tokenized"].values.tolist()
print(perplexity(wiki_sample_data,unigram_model))

1215.7165914041525


## TODO #2 **Please explain why this model give us such a high perplexity.**

**Your answer**:  Because this is just unigram model, it can tell only the possibillity of each word. Moreover, the model doesn't handle the unknown word properly.

# Bigram

Next, you will create a better language model than a unigram (which is not much to compare with). But first, it is very tedious to count every pair of words that occur in our corpus by ourselves. In this case, nltk provide us a simple library which will do it for us.

In [82]:
#example of nltk usage for bigram
sentence = 'I always search google for an answer .'

print('This is how nltk generate bigram.')
for w1,w2 in bigrams(sentence.split(), pad_right=True, pad_left=True):
    print (w1,w2)
print('None is used as a start and end of sentence symbol.')

This is how nltk generate bigram.
None I
I always
always search
search google
google for
for an
an answer
answer .
. None
None is used as a start and end of sentence symbol.


Now, you should be able to implement a bigram model by yourself. Also, you must create a new perplexity calculation for bigram. The result perplexity should be around 58.79 and 146.27 on train and test data.

## TODO #3 **Create a Bigram Model**

In [83]:
def getBigramModel(data):
    ###FILL YOUR CODE HERE###
    unigram_count = defaultdict(lambda: 0.0)
    for sentence in data:
        sentence += '|</s>'
        sentence = '<s>|' + sentence
        for word in sentence.split('|'):
            unigram_count[word] += 1
    bigram_count = defaultdict(lambda: defaultdict(lambda: 0.0))
    for sentence in data:
        sentence += '|</s>'
        sentence = '<s>|' + sentence
        words = sentence.split('|')
        for i in range(1,len(words)) :
            bigram_count[words[i]][words[i-1]] += 1
    model = defaultdict(lambda: defaultdict(lambda: 0.0))
    for x in bigram_count:
        for y in bigram_count[x]:
            model[x][y] = bigram_count[x][y] / unigram_count[y]
    return model

bigram_model = getBigramModel(train)


## TODO #4 **Calculate Perplexity for Bigram Model**



In [84]:
def calculate_sentence_ln_prob(sentence, model):
    sentence += '|</s>'
    sentence = '<s>|' + sentence
    words = sentence.split('|')
    ln_prob = .0
    for i in range(1,len(words)):
        ln_prob += getLnValue(model[words[i]][words[i-1]])
    return ln_prob, len(words) - 1

def perplexity(test,model):
    ln_prob = .0
    word_count = .0
    for sentence in test:
        ans = calculate_sentence_ln_prob(sentence, model)
        ln_prob += ans[0]
        word_count += ans[1]
    return math.exp(-ln_prob / word_count)



In [85]:
print(perplexity(train,bigram_model))
print(perplexity(test, bigram_model))

# 58.78942889767147
# 146.26539331038614

58.78942889767147
146.26539331038614


## MyCourseVille Question #2 
How much perplexity do you get with the Bigram model on the tokenized_wiki_sample dataset?

Please also leave the number below.

In [86]:
print(perplexity(wiki_sample_data,bigram_model))

744.3142671561902


# Smoothing

Usually any ngram models have a sparsity problem, which means it does not have every possible ngram of words in the dataset. Smoothing techniques can alleviate this problem. In this section, you will implement two basic smoothing methods laplace smoothing and interpolation for bigram.

## TODO #5 **Bigram with Laplace smoothing (Add-One Smoothing)**

In [87]:
#Laplace Smoothing
def getBigramWithLaplaceSmoothing(data):
    #Fill code here
    unigram_count = defaultdict(lambda: 0.0)
    for sentence in data:
        sentence += '|</s>'
        sentence = '<s>|' + sentence
        for word in sentence.split('|'):
            unigram_count[word] += 1
    bigram_count = defaultdict(lambda: defaultdict(lambda: 1.0))
    for sentence in data:
        sentence += '|</s>'
        sentence = '<s>|' + sentence
        words = sentence.split('|')
        for i in range(1,len(words)) :
            bigram_count[words[i]][words[i-1]] += 1
    model = defaultdict(lambda: defaultdict(lambda: math.pow(len(unigram_count), -1)))
    for x in bigram_count:
        for y in bigram_count[x]:
            model[x][y] = bigram_count[x][y] / (unigram_count[y] + len(unigram_count)-1) 
    return model


bigram_with_laplace_model = getBigramWithLaplaceSmoothing(train)
print(perplexity(train,bigram_with_laplace_model))
print(perplexity(test, bigram_with_laplace_model))

# 974.8134581679766
# 1098.1622194979489

974.8134581679766
1098.1760884387186


## MyCourseVille Question #3 
How much perplexity do you get with the Bigram model with Laplace Smoothing on the tokenized_wiki_sample dataset?

Please also leave the number below.

In [88]:
print(perplexity(wiki_sample_data,bigram_with_laplace_model))

3372.4691484637815


## TODO #6 **Bigram with Interpolation**
lambda value is 0.7 for bigram, 0.25 for unigram, and 0.05 for unknown word

In [89]:
#interpolation

#---------------This version same logic as lecture---------------
def getBigramWithInterpolation(data):
    #Fill code here
    unigram = getUnigramModel(data)
    bigram = getBigramModel(data)
    c = math.pow(len(unigram), -1)*0.05
    model = defaultdict(lambda: defaultdict(lambda: c)) # Remark1: set default to c 
    def check_unigram(word):
        p = 0.25*unigram[word] + c
        return lambda: p
    for x in unigram.keys():
        tmp = 0.25*unigram[x] + c
        model[x] = defaultdict(check_unigram(x))# Remark2: change defalt value to 0.25*unigram[x] + c if current word in unigram
        for y in bigram[x]:
            model[x][y] = 0.7*bigram[x][y] + 0.25*unigram[x] + c # Remark2: Adding value with 0.7*bigram[x][y] if current word and previous word are in bigram
    return model
#-----------------------------------------------------------------

#--------This version work for MyCourseVille and testcase--------
def getBigramWithInterpolation_forMCV(data):
    #Fill code here
    unigram = getUnigramModel(data)
    bigram = getBigramModel(data)
    c = math.pow(len(unigram),-1)
    model = defaultdict(lambda: defaultdict(lambda: 0.0)) 
    for x in unigram:
        model[x] = defaultdict(lambda: 0.0) 
        for y in bigram[x]:
            model[x][y] += 0.7*bigram[x][y] + 0.05*c + 0.25*unigram[x]

    return model
#-----------------------------------------------------------------

print("-------------exactly version-------------")
bigram_with_interpolation_model = getBigramWithInterpolation(train)
print(perplexity(train,bigram_with_interpolation_model))        
print(perplexity(test,bigram_with_interpolation_model))
print("-------------for MCV version-------------")
bigram_with_interpolation_model_forMCV = getBigramWithInterpolation_forMCV(train)
print(perplexity(train,bigram_with_interpolation_model_forMCV))        
print(perplexity(test,bigram_with_interpolation_model_forMCV))

# 73.38409869825665
# 172.67485908813356

-------------exactly version-------------
73.54392427626321
150.8055942710986
-------------for MCV version-------------
73.54392427626321
173.04288340104554


**Description for Bigram with interpolation model**

In bigram with interpolation model, I use dict of dict to create model. First index (model[x]) is for current word, and the second index[model[x][y]] for previous word.

At remark1, I set default value for unknown word to 0.25*(1/vocabulary count).

At remark2, If current word is one of the all vocabulary, the default value will change to 0.05*(1/vocabulary count) + 0.25*unigram[x].

At remark3, After I change default value in remark2, the value must add by 0.7*bigram[x][y] for the current word and previous word that in bigram.

## MyCourseVille Question #4 
How much perplexity do you get with the Bigram model with Interpolation on the tokenized_wiki_sample dataset?

Please also leave the number below.

In [64]:
print("-------------exactly version-------------") 
print(perplexity(wiki_sample_data,bigram_with_interpolation_model))
print("-------------for MCV version-------------")
print(perplexity(wiki_sample_data,bigram_with_interpolation_model_forMCV))

-------------exactly version-------------
838.6295860561921
-------------for MCV version-------------
821.2170455684426


# Language modeling on multiple domains

Sometimes, we do not have enough data to create a language model for a new domain. In that case, we can improvised by combining several models to improve result on the new domain.

In this exercise you will try to merge two language models from news and article domains to create a language model for the encyclopedia domain.

In [65]:
# create article data
encyclo_data=[]
fp= io.open('BEST2010/encyclopedia.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    encyclo_data.append(line.strip()[:-1])
fp.close()

article_data=[]
fp= io.open('BEST2010/article.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    article_data.append(line.strip()[:-1])
fp.close()

First, you should try to calculate perplexity of your bigram with interpolation using "news data" (train) on "encyclopedia data" (test). The result perplexity should be around 727.35.

For your information, a bigram model with interpolation using "article data" (train) to test on "encyclopedia data" (test) has a perplexity of 505.80.

In [66]:
# print perplexity of bigram with interpolation on article data        
# 928.8461907933413
print("-------------exactly version-------------") 
print(perplexity(encyclo_data,bigram_with_interpolation_model))
bigram_train_with_article = getBigramWithInterpolation(article_data)
print(perplexity(encyclo_data,bigram_train_with_article))
print("-------------for MCV version-------------")
print(perplexity(encyclo_data,bigram_with_interpolation_model_forMCV))
bigram_train_with_article_forMCV = getBigramWithInterpolation_forMCV(article_data)
print(perplexity(encyclo_data,bigram_train_with_article_forMCV))

-------------exactly version-------------
587.985335505008
440.2344588156499
-------------for MCV version-------------
728.8985182322733
519.5635034442969


## TODO #7 
Write a model that produce 450.0 or less perplexity on encyclopedia data without using data from the encyclopedia as training data. (Hint : Try to combine a model with news data and a model with article data together.)

In [67]:
# Fill code here
# 428.8525 (on combined data)
combined_data = best2010
for line in article_data:
    combined_data.append(line)
combined_model = getBigramWithInterpolation(combined_data)
print("-------------exactly version-------------") 
print('Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data',perplexity(encyclo_data, combined_model))
print("-------------for MCV version-------------")
combined_model_forMCV = getBigramWithInterpolation_forMCV(combined_data)
print('Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data',perplexity(encyclo_data, combined_model_forMCV))

-------------exactly version-------------
Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data 403.9378819091659
-------------for MCV version-------------
Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data 428.3308600402633


## TODO #8 
## Kneser-ney on "News"

<!-- Reimplement equation 4.33 in SLP textbook (https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf) -->

Implement Bigram Knerser-ney LM. The result perplexity should be around 75.51 and 183.35 on train and test data. 


In [68]:
# Fill codehere

#------------------------------------------
# create unigram & bigram counting tables
#------------------------------------------
# unigram_count = defaultdict(lambda: 0.0)
# bigram_count = defaultdict(lambda: 0.0)
# bigram_cn_count = defaultdict(lambda: 0.0)
# unigram_cn_count = defaultdict(lambda: 0.0)
# model = defaultdict(lambda: 0.0)

#------------------------------------------
# Kneserney
#------------------------------------------
# 1) counting all elements

# 2) compute p_cn (continuation prob)

# 3) compute p_kn (kneser-ney prob)

def getKneserNeyModel(data):
    unigram_count = defaultdict(lambda: 0.0)
    for sentence in data:
        sentence += '|</s>'
        sentence = '<s>|' + sentence
        for word in sentence.split('|'):
            unigram_count[word] += 1
    bigram_count = defaultdict(lambda: defaultdict(lambda: 0.0))
    for sentence in data:
        sentence += '|</s>'
        sentence = '<s>|' + sentence
        words = sentence.split('|')
        for i in range(1,len(words)) :
            bigram_count[words[i]][words[i-1]] += 1
    cn_count = 0.0  
    for x in bigram_count:
        cn_count += len(bigram_count[x])
    fw_count = defaultdict(lambda: 0.0)
    for x in bigram_count:
        fw_count[x] = len(bigram_count[x])
    nw_count = defaultdict(lambda: 0.0)
    nw_value = defaultdict(lambda: 0.0)
    nw_prob = defaultdict(lambda: 0.0)
    for x in bigram_count:
        for y in bigram_count[x]:
            nw_count[y] += 1
            nw_value[y] += bigram_count[x][y]
    for x in nw_count:
        nw_prob[x] = nw_count[x]/nw_value[x]
    model = defaultdict(lambda: defaultdict(lambda: 0.0))
    for x in bigram_count:
        for y in bigram_count[x]:
            model[x][y] = (max(0, bigram_count[x][y]-0.75) / unigram_count[y]) + (0.75*nw_prob[y]*fw_count[x]/cn_count)

    return model



KN_model = getKneserNeyModel(train)
print (perplexity(train,KN_model))        
print (perplexity(test,KN_model))

# 75.5096977631044
# 183.34798416569242

71.14054002208687
155.09274968738495


**Description for Bigram with Kneser-Ney model**

In this model I implement like interpolation model, but add more calculation model.

## MyCourseVille Question #5 
How much perplexity do you get with the Kneser-Ney model on the tokenized_wiki_sample dataset?

Please also leave the number below.

In [279]:
print (perplexity(wiki_sample_data,KN_model))

793.9339539916751
